In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## nn.Conv2d init

In [5]:
#export

from exp.nb_02 import *
def get_data():
    path = Path('/storage/data/mnist/mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s):
    return (x - m) / s

In [6]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      /opt/conda/envs/fastai/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [7]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [8]:
x_train.shape, x_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

In [9]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [10]:
n, *xdims = x_train.shape
c = len(y_train.unique())
nh = 32
n, c

(50000, 10)

In [11]:
l1 = nn.Conv2d(1, nh, 5)

In [12]:
x = x_valid[:100]

In [15]:
l1(x).shape

torch.Size([100, 32, 24, 24])

In [16]:
def stats(x): return x.mean(),x.std()

In [17]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [18]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0007, grad_fn=<MeanBackward0>),
  tensor(0.1151, grad_fn=<StdBackward0>)),
 (tensor(0.0502, grad_fn=<MeanBackward0>),
  tensor(0.1121, grad_fn=<StdBackward0>)))

In [19]:
t = l1(x)

In [20]:
stats(t)

(tensor(0.0514, grad_fn=<MeanBackward0>),
 tensor(0.5972, grad_fn=<StdBackward0>))

In [22]:
nn.init.kaiming_normal_(l1.weight, a=1.0)
stats(l1(x))

(tensor(0.0434, grad_fn=<MeanBackward0>),
 tensor(1.0763, grad_fn=<StdBackward0>))

In [23]:
import torch.nn.functional as F

In [25]:
def f1(x, a=0.):
    return F.leaky_relu(x, a)

In [27]:
nn.init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.3175), tensor(0.8203))

In [28]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.3175), tensor(0.8203))

In [29]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [30]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [32]:
nf, ni, *_ = l1.weight.shape
nf, ni 

(32, 1)

In [33]:
fan_in = ni * rec_fs
fan_out = nf * rec_fs

fan_in, fan_out

(25, 800)

In [34]:
def gain(a):
    return math.sqrt(
        2 / (1 + a **2)
    )

In [35]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)